In [ ]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

In [ ]:
batch_size = 128
num_classes = 10
epochs = 75

In [ ]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# Define teacher model

from keras.initializers import lecun_normal
from keras import models, layers

def build_teacher():
    initializer = lecun_normal(seed=1)
    
    teacher = Sequential()
    teacher.add(Conv2D(32, (3, 3),
                input_shape=x_train.shape[1:],
                kernel_initializer=initializer))
    teacher.add(Activation('relu'))
    teacher.add(MaxPooling2D(pool_size=(2, 2)))

    teacher.add(Conv2D(64, (3, 3),
                       kernel_initializer=initializer))
    teacher.add(Activation('relu'))
    teacher.add(MaxPooling2D(pool_size=(2, 2)))

    teacher.add(Flatten())
    teacher.add(Dense(256, kernel_initializer=initializer))
    teacher.add(Activation('relu'))
    teacher.add(Dense(num_classes, kernel_initializer=initializer))
    teacher.add(Activation('softmax'))
    
    return teacher

In [ ]:
# Compile and config data augmentation

teacher = build_teacher()

teacher.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])


datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format='channels_last',
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Train

from keras.callbacks import ReduceLROnPlateau, EarlyStopping

datagen.fit(x_train)
history = teacher.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                  epochs=epochs,
                                  validation_data=(x_test, y_test),
                                  callbacks=[
                                        EarlyStopping(monitor='val_loss', patience=5, min_delta=0.005)
                                    ])

In [ ]:
# Plot loss

import matplotlib.pyplot as plt

plt.plot(range(1, len(history.history['val_loss'])+1), history.history['val_loss'], label='val')
plt.plot(range(1, len(history.history['loss'])+1), history.history['loss'], label='training')
plt.title('Progression of loss on benchmark')
plt.xlabel('epoch');
plt.ylabel('hard logloss');
plt.legend()
plt.savefig('benchmark_logloss.png')

In [ ]:
results = teacher.evaluate(x_test, y_test)

In [ ]:
teacher.summary()

In [ ]:
print('Errors: ', len(x_test) - results[1] * len(x_test))
print('Accuracy: ', results[1]*100, '%')

In [ ]:
teacher.save('models/cifar10_teacher.h5')